In [1]:
### Baseline recommender - priporoči izdelke ki jih je kupec kupil en teden pred zadnim nakupom ###

In [2]:
#importi
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
from datetime import date, timedelta
import math

In [3]:
parse_dates = ['datum']
print("loading nakupi.csv...")
orders = pd.read_csv("Podatki/nakupi.csv", dtype={ #nrows=500000
        'kupec': np.uint32,
        'st_racuna': np.uint32,
        'artikel': np.uint32,
        'letak': np.bool},
        parse_dates=parse_dates)
print("done")

loading nakupi.csv...
done


In [4]:
#Preparing data sets train/test
print("Preparing data")
#max_datum = orders['datum'].max()            

tmp = pd.DataFrame()
orders['last_order_datum'] = orders.groupby('kupec')['datum'].transform(max)
tmp = orders.assign(last=orders.datum >= orders.last_order_datum - timedelta(7)) #zadnji teden
tmp = tmp.assign( manj_kot_14= tmp.datum >= orders.last_order_datum - timedelta(14) )
tmp = tmp.assign( vec_kot_7= tmp.datum < orders.last_order_datum - timedelta(7) )
tmp = tmp.assign( second_last= tmp.manj_kot_14 & tmp.vec_kot_7) #predzadnji teden

### TEST ###
test_second_last = tmp.loc[tmp['second_last'] == True] #  ostanejo samo predzadnji zadnji nakup ali ne
test_second_last = test_second_last.groupby('kupec').agg({'artikel':lambda x: list(x)}) # transforms real artikels to list

test_last = tmp.loc[tmp['last'] == True] # ostanejo samo zadni nakupi
test_for_f1 = test_last.groupby('kupec').agg({'artikel':lambda x: list(x), 'datum': 'first'}) # transforms real artikels to list
print("done")

Preparing data
done


In [5]:
### TESTING ####

def eval_fun(labels, preds):
    labels = labels
    preds = preds#.split(' ')
    rr = (np.intersect1d(labels, preds))
    if isinstance(preds, float) :
        return (0,0,0)
    precision = np.float(len(rr)) / len(preds)
    recall = np.float(len(rr)) / len(labels)
    try:
        f1 = 2 * precision * recall / (precision + recall)
        #print("   p={0:.2f} r={0:.2f} f={0:.2f} preds: ".format(precision, recall, f1, preds) + str(rr))
    except ZeroDivisionError:
        return (precision, recall, 0.0)
    return (precision, recall, f1)


###########
test_for_f1 = pd.merge(test_for_f1, test_second_last, how='left', on='kupec')
print("done")
len(test_for_f1)

done


19650

In [6]:
res = list()
for entry in test_for_f1.itertuples():
    #print("order:{} preds:{}".format(entry[1], entry[2]))
    res.append(eval_fun(entry[1], entry[3]))

res = pd.DataFrame(np.array(res), columns=['precision', 'recall', 'f1'])

In [7]:
res.describe()

,precision,recall,f1
count,19650.000000,19650.000000,19650.000000
mean,0.134892,0.100344,0.101473
std,0.166469,0.125508,0.114356
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.086957,0.062500,0.076070
75%,0.227273,0.166667,0.179487
max,1.000000,1.000000,1.000000


In [8]:
### F-Score samo z izdelki iz letaka ###

In [9]:
#naredimo dict z izdelki iz letaka
print('Making letak dict...')
letak_d= dict()
for row in orders.itertuples():
    datum = row.datum
    if row.letak == True:
        if datum.weekday() == 0:   #pon
            datum = datum - timedelta(4)
        elif datum.weekday() == 1: #tor
            datum = datum - timedelta(5)
        elif datum.weekday() == 2: #sre
            datum = datum - timedelta(6)
        elif datum.weekday() == 4: #pet
            datum = datum - timedelta(1)
        elif datum.weekday() == 5: #sob
            datum = datum - timedelta(2)
        elif datum.weekday() == 6: #ned
            datum = datum - timedelta(3)
        if datum.weekday() == 3:   #letak se začne s četrtkom
            if datum not in letak_d:
                letak_d[datum] = {row.artikel}
            else:
                letak_d[datum].add(row.artikel)
print('done')

Making letak dict...
done


In [10]:
def letak_eval_fun(labels, preds):
    rr = (np.intersect1d(labels, preds))
    precision = 0
    recall = 0
    try:
        precision = np.float(len(rr)) / len(preds)
        recall = np.float(len(rr)) / len(labels)
        f1 = 2 * precision * recall / (precision + recall)
        #print("   p={0:.2f} r={0:.2f} f={0:.2f} preds: ".format(precision, recall, f1, preds) + str(rr))
    except ZeroDivisionError:
        return (precision, recall, 0.0)
    return (precision, recall, f1)

In [11]:
res_letak = list()
for entry in test_for_f1.itertuples():
    datum = entry[2] #datum
    if datum.weekday() == 0:   #pon
        datum = datum - timedelta(4)
    elif datum.weekday() == 1: #tor
        datum = datum - timedelta(5)
    elif datum.weekday() == 2: #sre
        datum = datum - timedelta(6)
    elif datum.weekday() == 4: #pet
        datum = datum - timedelta(1)
    elif datum.weekday() == 5: #sob
        datum = datum - timedelta(2)
    elif datum.weekday() == 6: #ned
        datum = datum - timedelta(3)
    labels = entry[1]
    preds = entry[3]
    letak = list(letak_d[datum])
    labels_letak = (np.intersect1d(labels, letak))
    preds_letak = (np.intersect1d(preds, letak))

    res_letak.append(letak_eval_fun(labels_letak, preds_letak))

res_letak = pd.DataFrame(np.array(res_letak), columns=['precision', 'recall', 'f1'])

In [12]:
res_letak.describe()

,precision,recall,f1
count,19650.000000,19650.000000,19650.000000
mean,0.098746,0.081576,0.077564
std,0.240925,0.203369,0.182041
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000
